In [15]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

with open("input_16.txt", 'r') as f:
    data = f.read()


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
#data = [
#    [int(match.group(1)), list(map(int, match.group(2).split()))]
#    for line in data.split("\n")
#    if (match := re.match(r'^(\d+):\s*(\d+(?:\s+\d+)*)', line))
#]
#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]

#grid, moves = data.split("\n\n")
#moves = moves.replace("\n", "")

grid = {(x,y): str(val) for y, r in enumerate(data.split('\n')) for x, val in enumerate(r)}
grid
#data = [int(x) for x in data.split(" ")]


# Output the parsed data
#print(data)

helpers.visualize_dict_grid(grid) 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# . # . . . . . . . . . # . # . . . # . . . # . . . . . . . . . . . # . . . . . # . . . . . . . # . . . # . . . . . . . . . . . # . . . . . . . . . . . . . # . . . . . # . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . E #
# . # . # . # # # # # . # . # . # . # . # . # . # # # # # # # . # . # . # # # . # # # . # . # # # . # . # . # # # . # # # # # # # . # # # # # # # . # # # . # . # # # . # . # # # . # . # # # # # # # . # # # . # . # . # . # . # . # . # # # . # # # # # # # # # # # . # # # # # . # . #
# . # . # . . . . . # . . . # . # . . . # . . . . . . . # . . . # . . . # . . . . . . . # . . . . . . . . . . . . . . . . . # . . . . . # . . . . . # . # 

In [ ]:
#too slow
moves_dx = {"<": np.array([-1,0]),">": np.array([1,0]),"^": np.array([0,-1]),"v": np.array([0,1])} 

orig_location = [key for key, value in grid.items() if value == "S"][0]
visited = set()
lowest_found = 999999999999999999999999999999999999999999999999
def DFS(location, visited, heading, score):
    global lowest_found
    if tuple(location) in visited or score >= lowest_found:
        return
    #n_grid = grid.copy()
    #n_grid[tuple(orig_location)] = "."
    #n_grid[tuple(location)] = "@"
    #helpers.visualize_dict_grid(n_grid) 
    routes = []
    if grid[tuple(location)] == "E":
        if score < lowest_found:
            lowest_found = score
            print(lowest_found)
        return score
    visited.add(tuple(location))
    for new_heading, move in moves_dx.items():
        new_location = location + move
        if grid[tuple(new_location)] == "." or grid[tuple(new_location)] == "E":
            DFS_score = DFS(new_location, visited.copy(), new_heading, score + 1 + 1000 * (heading != new_heading))
            if DFS_score:
                routes.append(DFS_score)
    if routes:
        return min(routes)
    return 
DFS(orig_location, visited, ">", 0)

In [16]:
import heapq
import numpy as np

moves_dx = {"<": np.array([-1,0]), ">": np.array([1,0]), "^": np.array([0,-1]), "v": np.array([0,1])}
directions = ["^", ">", "v", "<"]

def turn_right(d):
    i = directions.index(d)
    return directions[(i + 1) % 4]

def turn_left(d):
    i = directions.index(d)
    return directions[(i - 1) % 4]

def neighbors(pos, d):
    fpos = tuple(np.array(pos) + moves_dx[d])
    yield (fpos, d)
    rdir = turn_right(d)
    rpos = tuple(np.array(pos) + moves_dx[rdir])
    yield (rpos, rdir)
    ldir = turn_left(d)
    lpos = tuple(np.array(pos) + moves_dx[ldir])
    yield (lpos, ldir)

def dijkstra(grid):
    start, end = None, None
    for k, v in grid.items():
        if v == "S":
            start = k
        elif v == "E":
            end = k

    start_state = (start, ">")
    dist = {start_state: 0}
    prev = {}
    heap = [(0, start_state)]

    while heap:
        cur_dist, current = heapq.heappop(heap)
        (cx, cy), cd = current
        if (cx, cy) == end:
            break
        if cur_dist > dist.get(current, float('inf')):
            continue
        for (nx, ny), nd in neighbors((cx, cy), cd):
            if (nx, ny) in grid and grid[(nx, ny)] != "#":
                if not cd == nd:
                    ndist = cur_dist + 1001
                else:
                    ndist = cur_dist + 1
                nxt = ((nx, ny), nd)
                if ndist < dist.get(nxt, float('inf')):
                    dist[nxt] = ndist
                    prev[nxt] = current
                    heapq.heappush(heap, (ndist, nxt))

    end_states = [s for s in dist if s[0] == end]
    if not end_states:
        return []
    end_state = min(end_states, key=lambda s: dist[s])
    print(dist[end_state])
    path = []
    cur = end_state
    while cur in prev:
        path.append(cur[0])
        cur = prev[cur]
    path.append(start)
    path.reverse()
    #print(ndist)
    return path

dijkstra(grid)


94444


[(1, 139),
 (2, 139),
 (3, 139),
 (4, 139),
 (5, 139),
 (6, 139),
 (7, 139),
 (8, 139),
 (9, 139),
 (9, 138),
 (9, 137),
 (8, 137),
 (7, 137),
 (7, 136),
 (7, 135),
 (7, 134),
 (7, 133),
 (7, 132),
 (7, 131),
 (7, 130),
 (7, 129),
 (7, 128),
 (7, 127),
 (7, 126),
 (7, 125),
 (7, 124),
 (7, 123),
 (7, 122),
 (7, 121),
 (6, 121),
 (5, 121),
 (4, 121),
 (3, 121),
 (3, 120),
 (3, 119),
 (2, 119),
 (1, 119),
 (1, 118),
 (1, 117),
 (2, 117),
 (3, 117),
 (4, 117),
 (5, 117),
 (5, 116),
 (5, 115),
 (6, 115),
 (7, 115),
 (7, 114),
 (7, 113),
 (7, 112),
 (7, 111),
 (7, 110),
 (7, 109),
 (6, 109),
 (5, 109),
 (5, 108),
 (5, 107),
 (5, 106),
 (5, 105),
 (5, 104),
 (5, 103),
 (5, 102),
 (5, 101),
 (5, 100),
 (5, 99),
 (5, 98),
 (5, 97),
 (5, 96),
 (5, 95),
 (5, 94),
 (5, 93),
 (5, 92),
 (5, 91),
 (5, 90),
 (5, 89),
 (4, 89),
 (3, 89),
 (3, 88),
 (3, 87),
 (2, 87),
 (1, 87),
 (1, 86),
 (1, 85),
 (1, 84),
 (1, 83),
 (2, 83),
 (3, 83),
 (4, 83),
 (5, 83),
 (5, 82),
 (5, 81),
 (4, 81),
 (3, 81),
 (2, 8

In [ ]:
#94445 too high off by one, sigh